# Preparing Snowflake
This note book will guide you through the required steps thats need to prepare your Snowflake account to deploy the demo [Streamlit ML App](https://github.com/kameshsampath/st-ml-app)

Typically we wil 

- [x] Create Schemas and Tables
- [x] Create a external stage to S3
- [x] Load Penguins Dataset on the table

## Schemas and Stages

In the next cell we will create the schemas and stages that will be used in this tutorial.

### Schemas

|Schema | Use|
|------- |----------------|
| apps | Will hold all applications e.g. Streamlit|
| data | Will hold all data tables  |
| stages | All  stages |
| file_formats | All  file formats that will be used during data load |



In [ ]:
-- data schema
CREATE SCHEMA IF NOT EXISTS DATA;
-- create schema to hold all stages
CREATE SCHEMA IF NOT EXISTS STAGES;
-- create schema to hold all file formats
CREATE SCHEMA IF NOT EXISTS FILE_FORMATS;
-- apps to hold all streamlit apps
CREATE SCHEMA IF NOT EXISTS APPS;


### Stages and File Format

We will create stage named `stages.st_ml_app_penguins` which will point to an s3 bucket `s3://sfquickstarts/misc` and it will use the file file format `file_formats.csv` to parse and laod CSV files.

In [ ]:
-- add an external stage to a s3 bucket
CREATE STAGE IF NOT EXISTS STAGES.ST_ML_APP_PENGUINS
  URL='s3://sfquickstarts/misc';

-- default CSV file format and allow values to quoted by "
CREATE FILE FORMAT IF NOT EXISTS FILE_FORMATS.CSV
  TYPE='CSV'
  SKIP_HEADER=1
  FIELD_OPTIONALLY_ENCLOSED_BY = '"';

## Load 

As part of next step let us load the data on `data.penguins` table using the file from `@stages.st_ml_app_penguins/penguins_cleaned.csv`

In [ ]:
-- Create table to hold penguins data
CREATE OR ALTER TABLE DATA.PENGUINS(
   SPECIES STRING NOT NULL,
   ISLAND STRING NOT NULL,
   BILL_LENGTH_MM NUMBER NOT NULL,
   BILL_DEPTH_MM NUMBER NOT NULL,
   FLIPPER_LENGTH_MM NUMBER NOT NULL,
   BODY_MASS_G NUMBER NOT NULL,
   SEX STRING NOT NULL
);

-- Load the data from penguins_cleaned.csv
COPY INTO DATA.PENGUINS
FROM @STAGES.ST_ML_APP_PENGUINS/PENGUINS_CLEANED.CSV
FILE_FORMAT=(FORMAT_NAME='FILE_FORMATS.CSV');

Let us select and verify the data,

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()
df = session.table('st_ml_app.data.penguins')
df.show()